<a href="https://colab.research.google.com/github/Eshita781996/ReadWrite/blob/main/ReadWrite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [2]:
import findspark
findspark.init()

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [4]:
# @title Read from CSV
df =  spark.read.option("header", "true").csv('/content/sample_data/california_housing_test.csv')
df.show(2)

+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population|households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|-122.050000|37.370000|         27.000000|3885.000000|    661.000000|1537.000000|606.000000|     6.608500|     344700.000000|
|-118.300000|34.260000|         43.000000|1510.000000|    310.000000| 809.000000|277.000000|     3.599000|     176500.000000|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
only showing top 2 rows



In [5]:
# @title Read from JSON
df =  spark.read.option("header", "true").format('json').option("multiline","true") .load('/content/sample_data/anscombe.json')
df.show(2)

+------+----+----+
|Series|   X|   Y|
+------+----+----+
|     I|10.0|8.04|
|     I| 8.0|6.95|
+------+----+----+
only showing top 2 rows



In [38]:

schema = {'emp_id':'string','emp_name':'integer','emp_city':'string','emp_salary':'integer'}
print(schema)

schema_of_csv = list()
for i in schema:
  schema_of_csv.append({'name': i,\
                       'type': schema[i],\
                       'nullable': False,\
                       'metadata':{}})

print(schema_of_csv)


{'emp_id': 'string', 'emp_name': 'integer', 'emp_city': 'string', 'emp_salary': 'integer'}
[{'name': 'emp_id', 'type': 'string', 'nullable': False, 'metadata': {}}, {'name': 'emp_name', 'type': 'integer', 'nullable': False, 'metadata': {}}, {'name': 'emp_city', 'type': 'string', 'nullable': False, 'metadata': {}}, {'name': 'emp_salary', 'type': 'integer', 'nullable': False, 'metadata': {}}]


In [39]:
schema = StructType.fromJson({'type':'struct','fields':schema_of_csv})
print(schema)

StructType(List(StructField(emp_id,StringType,false),StructField(emp_name,IntegerType,false),StructField(emp_city,StringType,false),StructField(emp_salary,IntegerType,false)))


In [47]:
df = spark.read.schema(schema).option("mode", "DROPMALFORMED").option('header','true').csv('/content/sample_data/sample.csv')
df.show()
df1 = spark.read.schema(schema).option("mode", "PERMISSIVE").option('header','true').csv('/content/sample_data/sample.csv')
df1.show()
#df2 = spark.read.schema(schema).option("mode", "FAILFAST").option('header','true').csv('/content/sample_data/sample.csv')
#df2.show()
print(df.dtypes)
print(df.columns)

+------+--------+--------+----------+
|emp_id|emp_name|emp_city|emp_salary|
+------+--------+--------+----------+
+------+--------+--------+----------+

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     1|    null|   Sydney|     35000|
|     2|    null|Melbourne|     45000|
|     3|    null|   Sydney|     55000|
+------+--------+---------+----------+

[('emp_id', 'string'), ('emp_name', 'int'), ('emp_city', 'string'), ('emp_salary', 'int')]
['emp_id', 'emp_name', 'emp_city', 'emp_salary']


In [42]:
employees_delta_df = spark.createDataFrame([(2, 'Peter', 'Melbourne', 55000.00),(5, 'Jessie', 'Brisbane', 42000.00)],
                                           schema='emp_id int,emp_name string,emp_city string,emp_salary float')
print(employees_delta_df.dtypes)
print(employees_delta_df.columns)

[('emp_id', 'int'), ('emp_name', 'string'), ('emp_city', 'string'), ('emp_salary', 'float')]
['emp_id', 'emp_name', 'emp_city', 'emp_salary']
